## Imports

In [1]:
%load_ext autoreload
%autoreload 2

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer

from dataset.aml_data_module import AMLDataModule
from model.aml_classifier import AMLClassifier

/mnt/d/coding/git/TomaszKaleczyc/AML_news_detection/environment/aml_venv/lib/python3.6/site-packages/pytorch_lightning/metrics/__init__.py:44: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  "`pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package"


## Model training

Model hyperparameters:

In [2]:
NUM_EPOCHS = 2
SEQUENCE_LENGTH = 200
SEQUENCE_OVERLAP = 50
BATCH_SIZE = 1  # TODO: at the moment model can only handle online learning


data_module = AMLDataModule(
    sequence_length=SEQUENCE_LENGTH,
    overlap=SEQUENCE_OVERLAP
)
model = AMLClassifier()
callbacks = [
    ModelCheckpoint(
        filename=f'AMLClassifier-seq_len{SEQUENCE_LENGTH}-ovlp{SEQUENCE_OVERLAP}'+'-{epoch}-{validation/accuracy:.3f}',
        monitor='validation/accuracy',
        mode='max',
        save_top_k=1,
        verbose=True,
    )
]
trainer = Trainer(
    max_epochs=NUM_EPOCHS,
    fast_dev_run=False,
    default_root_dir='../output',
    callbacks=callbacks
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/mnt/d/coding/git/TomaszKaleczyc/AML_news_detection/environment/aml_venv/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initializat

Training loop:

In [3]:
trainer.fit(
    model,
    train_dataloader=data_module.train_dataloader(),
    val_dataloaders=data_module.val_dataloader()
)

Tokenisation of content strips:


  0%|          | 0/30 [00:00<?, ?it/s]

Loaded dataset train
Number of tokens: 30
Number of classes: 30
Tokenisation of content strips:


  0%|          | 0/15 [00:00<?, ?it/s]


  | Name                | Type       | Params
---------------------------------------------------
0 | feature_extractor   | BertModel  | 108 M 
1 | aggregating_network | LSTM       | 348 K 
2 | predictor           | Sequential | 3.1 K 
3 | accuracy            | Accuracy   | 0     
---------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
434.646   Total estimated model params size (MB)


...

Validation sanity check: 0it [00:00, ?it/s]

/mnt/d/coding/git/TomaszKaleczyc/AML_news_detection/environment/aml_venv/lib/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:103: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f'The dataloader, {name}, does not have many workers which may be a bottleneck.'


val/accuracy tensor(0.)


/mnt/d/coding/git/TomaszKaleczyc/AML_news_detection/environment/aml_venv/lib/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:103: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f'The dataloader, {name}, does not have many workers which may be a bottleneck.'


Training: 0it [00:00, ?it/s]